In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/DACON_PakChoi

/content/drive/MyDrive/Colab Notebooks/DACON_PakChoi


해야할 것
1. target 변화
2. 모델 변화

In [ ]:
import pandas as pd
import os
from sklearn.metrics import r2_score
'''
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import AdaBoostClassifier
'''

from sklearn.ensemble import HistGradientBoostingClassifier


from sklearn.ensemble import VotingClassifier

# 실패
'''
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF #위랑같이씀
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.gaussian_process import GaussianProcessClassifier #5분째돌길래 포기
from sklearn.neighbors import RadiusNeighborsClassifier
'''

'\nfrom sklearn.semi_supervised import LabelPropagation\nfrom sklearn.semi_supervised import LabelSpreading\nfrom sklearn.linear_model import LogisticRegressionCV\nfrom sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis\nfrom sklearn.neighbors import RadiusNeighborsClassifier\nfrom sklearn.linear_model import RidgeClassifierCV\nfrom sklearn.svm import NuSVC\nfrom sklearn.gaussian_process import GaussianProcessClassifier\nfrom sklearn.gaussian_process.kernels import RBF #위랑같이씀\nfrom sklearn.ensemble import GradientBoostingClassifier \nfrom sklearn.gaussian_process import GaussianProcessClassifier #5분째돌길래 포기\nfrom sklearn.neighbors import RadiusNeighborsClassifier\n'

train dataset 만들기

In [ ]:
train_x = pd.DataFrame()
for filenum in [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26]:
  if filenum < 10:
    tmp_df= pd.read_csv(f'./train_input/CASE_0{filenum}.csv')
  else:
    tmp_df= pd.read_csv(f'./train_input/CASE_{filenum}.csv')

  tmp_df.fillna(value=0,inplace=True)
  tmp_df['시간'] = pd.to_datetime(tmp_df['시간'])
  tmp_df['date'] = tmp_df['시간'].dt.date
  tmp_df = tmp_df.groupby('date').sum()

  train_x = pd.concat([train_x, tmp_df])

In [ ]:
train_y = pd.DataFrame()
for filenum in [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26]:
  if filenum < 10:
    tmp_df= pd.read_csv(f'./train_target/CASE_0{filenum}.csv')
  else:
    tmp_df= pd.read_csv(f'./train_target/CASE_{filenum}.csv')

  absolute_value=1
  list_tmp_df=[]

  tmp_df["rate+1"]= tmp_df['rate']+1

  for i in list(tmp_df["rate+1"]):
      absolute_value*=i
      list_tmp_df.append(absolute_value)
  tmp_df['rate_based_on_init']=list_tmp_df
  tmp_df['tmp']=[1]+list_tmp_df[:-1]
  tmp_df['value']=tmp_df['rate_based_on_init']-tmp_df['tmp']

  train_y = pd.concat([train_y, tmp_df])

train dataset 훈련 : 이 부분 바꿔주면 돼

In [ ]:
from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(train_y['rate_based_on_init'])

In [ ]:
clf1 = HistGradientBoostingClassifier(min_samples_leaf=1) # scikit run에 있는 모델들로 바꿔보기
clf1.fit(train_x, y_transformed) # rate+1 부분을 rate/rate+1/rate_based_on_init/value 중의 하나로

HistGradientBoostingClassifier(min_samples_leaf=1)

test dataset 만들기

In [ ]:
test_x = pd.DataFrame()
for filenum in [27, 28, 29, 32, 33, 35]:
  tmp_df= pd.read_csv(f'./train_input/CASE_{filenum}.csv')

  tmp_df.fillna(value=0,inplace=True)
  tmp_df['시간'] = pd.to_datetime(tmp_df['시간'])
  tmp_df['date'] = tmp_df['시간'].dt.date
  tmp_df = tmp_df.groupby('date').sum()

  test_x = pd.concat([test_x, tmp_df])

In [ ]:
test_y = pd.DataFrame()
for filenum in [27, 28, 29, 32, 33, 35]:
  tmp_df= pd.read_csv(f'./train_target/CASE_{filenum}.csv')

  absolute_value=1
  list_tmp_df=[]

  tmp_df["rate+1"]= tmp_df['rate']+1

  for i in list(tmp_df["rate+1"]):
      absolute_value*=i
      list_tmp_df.append(absolute_value)
  tmp_df['rate_based_on_init']=list_tmp_df
  tmp_df['tmp']=[1]+list_tmp_df[:-1]
  tmp_df['value']=tmp_df['rate+1']-tmp_df['tmp']

  test_y = pd.concat([test_y, tmp_df])

test dataset 검증 : 이 부분 바꿔주면 돼

In [ ]:
r2 = r2_score(clf1.predict(test_x), test_y['rate_based_on_init']) # rate+1 부분을 rate/rate+1/rate_based_on_init/value 중의 하나로
print(r2)

-17.681167150672028
